# Librerias 

In [6]:
import pandas as pd
import psycopg2
from psycopg2 import sql
from datetime import datetime

In [ ]:
pip install sqlalchemy psycopg2 pandas 

## Configuración de conexión a BD y paths arcvhivos  

In [8]:
# Configuración de conexión
DB_CONFIG = {
    'host': 'localhost',
    'port': '5432',
    'database': 'ingicat_db',
    'user': 'fabian',
    'password': 'ingicat*1003'
}

# Ruta al archivo Excel
EXCEL_PATH = '20250404_BASE DOCUMENTAL_PREDIOS PROPIOS.xlsx'
SHEET_NAME = 'T_DOCUM'
TABLE_NAME = 'task_predio'  # tabla de tu modelo Predio


## leer archivo de excel

In [9]:
try:
    df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME, header=1)  
    print("Columnas disponibles en el Excel:", df.columns.tolist())
    print(f"Archivo leído correctamente. Filas: {len(df)}")
except Exception as e:
    print(f"Error leyendo el archivo Excel: {e}")
    exit(1)

print(df.info())

/home/fabian/desarrollo/migration/myenv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Columnas disponibles en el Excel: ['PROYECTO', 'ID', 'VIGENCIA', 'VICEPRESIDENCIA', 'GERENCIA', 'CAMPO', 'CODIGO SIG DE ESTUDIO', 'CATEGORÍA DEL PREDIO - FMI', 'ESTADO DEL FOLIO DE MATRÍCULA INMOBILIARIA', 'FOLIO DE MATRÍCULA INMOBILIARIA', 'CÉDULA CATASTRAL SEGÚN FMI', 'NOMBRE DEL PREDIO SEGÚN FMI', '# DE ANOTACIÓN', 'CATEGORÍA FMI (TRADICIÓN - COMPLEMENTACIÓN)', 'TIPODOCUMENTAL', 'NÚMERO DE DOCUMENTO', 'FECHA DEL DOCUMENTO (DD/MM/AAAA)', 'NOTARÍA / ESTABLECIMIENTO', 'CIUDAD O MUNICIPIO', 'TIPO ACTO', 'DOCUMENTOS + MUNICIPIO', 'NOMBRE PREDIO - OPENTEXT', 'CODIGO SIG - OPENTEXT', 'ESTADO', 'CODIGO SIG - ASOCIADO', 'ESTADO COMPRA', 'SUB-ESTADO DE COMPRA', 'FECHA DE SOLICITUD ', 'REITERACION', 'FECHA RESPUESTA ENTIDAD', 'FECHA DE PAGO', 'VALOR DEL PAGO', 'FECHA DE ADQUISICIÓN', 'ESTRATEGIA DE CONSECUCION', 'FECHA ULTIMA REITERACION', 'RESPONSABLE DE LA GESTIÓN', 'ENVÍO OPEN TEXT', 'FECHA ENVÍO GESTIÓN DOCUMENTAL', 'ACCIÓN TECNICA', 'LINK SOPORTE ENVIO GESTION DOCUMENTAL', 'LINK SHAREPOIN

## ajuste del dataframe

### cargar los headers de la tabla de la base de datos

In [10]:
from sqlalchemy import create_engine, inspect
engine = create_engine(
    f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)


inspector = inspect(engine)
print(inspector.get_table_names())

columns = inspector.get_columns(TABLE_NAME)
for col in columns:
    print(col['name'], col['type'])


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

### Renombrar los campos de acuerdo a los de la base de datos

In [16]:
db_columns = [
    "id", "proyecto", "vigencia", "gerencia", "categoria_predio_fmi", "estado_folio_matricula",
    "categoria_fmi", "tipo_documental", "estado", "estado_compra", "sub_estado_compra",
    "envio_open_text", "accion_tecnica", "user_id", "datecompleted", "fecha_respuesta",
    "fecha_solicitud", "es_importante", "ultima_fecha_acceso", "campo", "ced_catastral",
    "cod_sig", "documento", "entidad", "fecha_documento", "fecha_reiteracion", "fmi",
    "municipio", "nom_predio", "adquirir", "cod_especificacion", "cod_sig_asociado",
    "cod_sig_opentext", "documentos_municipio", "estrategia", "fecha_adquisicion",
    "fecha_nueva_busqueda", "fecha_pago", "link_sharepoint", "nombre_predio_opentext",
    "paquete", "repetido", "responsable_adquisicion", "responsable_nueva_busqueda",
    "responsable_seguimiento", "valor_pago"
]

rename_dict = {
    'PROYECTO': 'proyecto',
    'ID': 'id',
    'VIGENCIA': 'vigencia',
    'GERENCIA': 'gerencia',
    'CAMPO': 'campo',
    'CATEGORÍA DEL PREDIO - FMI': 'categoria_predio_fmi',
    'ESTADO DEL FOLIO DE MATRÍCULA INMOBILIARIA': 'estado_folio_matricula',
    'CATEGORÍA FMI (TRADICIÓN - COMPLEMENTACIÓN)': 'categoria_fmi',
    'TIPODOCUMENTAL': 'tipo_documental',
    'ESTADO': 'estado',
    'ESTADO COMPRA': 'estado_compra',
    'SUB-ESTADO DE COMPRA': 'sub_estado_compra',
    'ENVÍO OPEN TEXT': 'envio_open_text',
    'ACCIÓN TECNICA': 'accion_tecnica',
    'FECHA RESPUESTA ENTIDAD': 'fecha_respuesta',
    'FECHA DE SOLICITUD ': 'fecha_solicitud',
    'ES IMPORTANTE': 'es_importante',
    'FECHA ULTIMA REITERACION': 'fecha_reiteracion',
    'CÉDULA CATASTRAL SEGÚN FMI': 'ced_catastral',
    'CODIGO SIG DE ESTUDIO': 'cod_sig',
    'NÚMERO DE DOCUMENTO': 'documento',
    'NOTARÍA / ESTABLECIMIENTO': 'entidad',
    'FECHA DEL DOCUMENTO (DD/MM/AAAA)': 'fecha_documento',
    'FMI': 'fmi',
    'CIUDAD O MUNICIPIO': 'municipio',
    'NOMBRE DEL PREDIO SEGÚN FMI': 'nom_predio',
    'ADQUIRIR': 'adquirir',
    'COD ESPECIFICACION': 'cod_especificacion',
    'CODIGO SIG - ASOCIADO': 'cod_sig_asociado',
    'CODIGO SIG - OPENTEXT': 'cod_sig_opentext',
    'DOCUMENTOS + MUNICIPIO': 'documentos_municipio',
    'ESTRATEGIA DE CONSECUCION': 'estrategia',
    'FECHA DE ADQUISICIÓN': 'fecha_adquisicion',
    'FECHA NUEVA BUSQUEDA': 'fecha_nueva_busqueda',
    'FECHA DE PAGO': 'fecha_pago',
    'LINK SHAREPOINT DEL DOCUMENTO': 'link_sharepoint',
    'NOMBRE PREDIO - OPENTEXT': 'nombre_predio_opentext',
    'PAQUETE': 'paquete',
    'Repetido': 'repetido',
    'RESPONSABLE DE LA GESTIÓN': 'responsable_adquisicion',
    'RESPONSABLE NUEVA BUSQUEDA': 'responsable_nueva_busqueda',
    'RESPONSABLE': 'responsable_seguimiento',
    'VALOR DEL PAGO': 'valor_pago'
}
# Renombrar columnas
df_renamed = df.rename(columns=rename_dict)
df_renamed.columns = df_renamed.columns.str.strip()

# Filtrar columnas válidas
df_filtrado = df_renamed[[col for col in df_renamed.columns if col in db_columns]]
print("Columnas finales:", df_filtrado.columns)

# Convertir fechas
fecha_cols = [
    'fecha_documento', 'fecha_solicitud', 'fecha_reiteracion',
    'fecha_respuesta', 'fecha_pago', 'fecha_adquisicion', 'fecha_nueva_busqueda'
]

for col in fecha_cols:
    if col in df_filtrado.columns:
        # Intentar convertir a datetime
        df_filtrado[col] = pd.to_datetime(df_filtrado[col], errors='coerce', dayfirst=True)
        print(f"{col}: {df_filtrado[col].dtype}")

        if pd.api.types.is_datetime64_any_dtype(df_filtrado[col]):
            # Convertir a formato 'YYYY-MM-DD' como string
            df_filtrado[col] = df_filtrado[col].dt.strftime('%Y-%m-%d')
        else:
            print(f"⚠️ La columna '{col}' no se convirtió correctamente a datetime")
        

# Convertir a booleano
def convertir_a_booleano(valor):
    if isinstance(valor, str):
        valor = valor.strip().upper()
        if valor == 'SI':
            return True
        elif valor == 'NO':
            return False
    return None  # o False, según lo que necesites

if 'adquirir' in df_filtrado.columns:
    df_filtrado['adquirir'] = df_filtrado['adquirir'].apply(convertir_a_booleano)
print(df_filtrado['valor_pago'].unique())
campos_numericos = ['valor_pago', 'cod_especificacion']
for col in campos_numericos:
    df_filtrado[col] = pd.to_numeric(df_filtrado[col], errors='coerce')  
print(df_filtrado['valor_pago'].unique())


for col in df_filtrado.columns:
    print(f"\nColumna: {col}")
    print("Valores únicos:", df_filtrado[col].unique())


Columnas finales: Index(['proyecto', 'id', 'vigencia', 'gerencia', 'campo', 'cod_sig',
       'categoria_predio_fmi', 'estado_folio_matricula', 'ced_catastral',
       'nom_predio', 'categoria_fmi', 'tipo_documental', 'documento',
       'fecha_documento', 'entidad', 'municipio', 'documentos_municipio',
       'nombre_predio_opentext', 'cod_sig_opentext', 'estado',
       'cod_sig_asociado', 'estado_compra', 'sub_estado_compra',
       'fecha_solicitud', 'fecha_respuesta', 'fecha_pago', 'valor_pago',
       'fecha_adquisicion', 'estrategia', 'fecha_reiteracion',
       'responsable_adquisicion', 'envio_open_text', 'accion_tecnica',
       'link_sharepoint', 'responsable_seguimiento',
       'responsable_nueva_busqueda', 'fecha_nueva_busqueda',
       'cod_especificacion', 'adquirir', 'repetido', 'paquete'],
      dtype='object')
fecha_documento: datetime64[ns]
fecha_solicitud: datetime64[ns]
fecha_reiteracion: datetime64[ns]
fecha_respuesta: datetime64[ns]
fecha_pago: datetime64[ns]
fe

/tmp/ipykernel_7757/658695117.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado[col] = pd.to_datetime(df_filtrado[col], errors='coerce', dayfirst=True)
/tmp/ipykernel_7757/658695117.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado[col] = df_filtrado[col].dt.strftime('%Y-%m-%d')
/tmp/ipykernel_7757/658695117.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

### enviando como in insert into a la tabla de la bd

In [35]:
from sqlalchemy.exc import SQLAlchemyError

errores = []

for i, row in df_filtrado.iterrows():
    try:
        # Convertir la fila a un DataFrame de una sola fila
        row_df = pd.DataFrame([row])

        # Insertar la fila
        row_df.to_sql(TABLE_NAME, con=engine, if_exists='append', index=False)
    
    except SQLAlchemyError as e:
        # Capturar el ID del registro (si tiene) y el error
        registro_id = row.get('id', f'fila_{i}')
        errores.append({'id': registro_id, 'error': str(e)})

# Mostrar resumen
print(f"Total de registros cargados exitosamente: {len(df_filtrado) - len(errores)}")
print(f"Total de errores: {len(errores)}")

# Convertir a DataFrame y guardar si hay errores
if errores:
    errores_df = pd.DataFrame(errores)
    errores_df.to_excel('errores_registros.xlsx', index=False)
    print("Se guardaron los errores en 'errores_registros.xlsx'")


Total de registros cargados exitosamente: 0
Total de errores: 7168
Se guardaron los errores en 'errores_registros.xlsx'
